In [ ]:
import re
import json
import os
from PyPDF2 import PdfReader

pdf_path = r'C:/Workspace/BTM_25Q3_GEN/_etc/rawdata/국어 지식 기반 생성(RAG) 참조 문서.pdf'
output_dir = 'docs'
os.makedirs(output_dir, exist_ok=True)

# PDF에서 전체 텍스트 추출
reader = PdfReader(pdf_path)
text = ""
for page in reader.pages:
    text += page.extract_text() + "/n"

# <> 단위로 제목과 본문 분리
pattern = r'<([^<>]+)>(.*?)((?=<[^<>]+>)|$)'  # <>로 감싸진 제목과 그 뒤의 본문
matches = re.findall(pattern, text, re.DOTALL)

for match in matches:
    title = match[0].strip()  # <> 내부 텍스트
    content = match[1].strip()  # <> 뒤 본문
    # 파일명, id 추출 (파일명-조항명 구조라면)
    if '-' in title:
        file_name, clause = map(str.strip, title.split('-', 1))
    else:
        file_name, clause = "output", title
    # json 데이터 생성
    data = {
        "id": clause,
        "content": content
    }
    # 파일 저장 (여러 조항이 한 파일에 들어갈 수 있음)
    json_path = os.path.join(output_dir, f"{file_name}.json")
    if os.path.exists(json_path):
        with open(json_path, 'r', encoding='utf-8') as f:
            arr = json.load(f)
    else:
        arr = []
    arr.append(data)
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(arr, f, ensure_ascii=False, indent=2)